In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import torch
from PIL import Image
from torchvision import transforms
import imgproc
from torchvision.transforms.functional import InterpolationMode as IMode
from Model.resnet import MyGenerator
from Model.aetad import AETAD_4Direct
import torch.nn.functional as F
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
model = MyGenerator()
checkpoint = torch.load('./jupyter_pth/my_down_up/g-last.pth', map_location='cuda:0')
checkpoint = {k.replace('module.',''):v for k,v in checkpoint.items()}
# model = torch.nn.DataParallel(model)
model.load_state_dict(checkpoint)
model.eval()

MyGenerator(
  (conv_block0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): PReLU(num_parameters=1)
  )
  (down_trunk): Sequential(
    (0): ResidualConvBlock(
      (rcb): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualConvBlock(
      (rcb): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [ ]:
vc = cv2.VideoCapture('./jupyter_src/vd/Pexels Videos 1812902.mp4')
frames = []
suc, frame = vc.read()
while suc:
    frames.append(frame)
    suc, frame = vc.read()

print(len(frames))

In [6]:
hr_frame = []
for frame in frames:
    input_t = imgproc.image2tensor(frame, range_norm=False, half=False)
    input_t = input_t.reshape(1, input_t.shape[0], input_t.shape[1], input_t.shape[2])
    with torch.no_grad():
        hr = model.forward(input_t)
        hr_img = imgproc.tensor2image(hr, range_norm=False, half=False)
        hr_frame.append(hr_img)

fps = len(frames) // 60
print(hr_frame[0].size)
img_size = (640 , 360 )
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
video_path = '../big_bunk_bunny_hr.avi'
videoWriter = cv2.VideoWriter(video_path,fourcc,fps,img_size)

for index,frame in enumerate(hr_frame):
    videoWriter.write(frame) # 图片写入视频

videoWriter.release()
print("finish")

691200
finish
